In [1]:
using Symbolics
function integrateTaylorPolynomials(eq::Num, x::Num)
    # this function works only for (positive) polynomials 
    #
    # the function needs only Symbolics.jl
    #
    # Since the Symbolics.coeff does not support expression with division, lowestNegativeOrder::Int option should be implemented afterwards

    lowestNegativeOrder::Int=0
    eq = simplify(eq;expand=true,polynorm=true)
    eqval=Symbolics.value(eq)
    

    # The above statement is to switch off the negative orders integral
   
    if Symbolics.isdiv(eqval)
        eq=eqval.num 
        denominator=eqval.den 
    else 
        denominator =1
    end
    highestOrder=Symbolics.degree(eq,x)
   
    tmpeq = eq
    neweq = 0
    if highestOrder>0
        for i in 1:highestOrder
            old_coef=Symbolics.coeff(eq,x^i)
            tmpeq-=old_coef*x^i
            new_coef=old_coef//(i+1)
            neweq+=new_coef*x^(i+1)        
        end
    end
    
    if lowestNegativeOrder<0
        old_coef=Symbolics.coeff(eq,x^(-1))
        tmpeq-=old_coef*x^(-1)
        new_coef=old_coef
        neweq+=new_coef*log(x)

        if lowestNegativeOrder<1
            for i in lowestNegativeOrder:-2
                old_coef=Symbolics.coeff(eq,x^i)
                tmpeq-=old_coef*x^i
                new_coef=old_coef//(i+1)
                neweq+=new_coef*x^(i+1)
            end
        end
    end

    # the rest of tmpeq should be the constant
    neweq+=tmpeq*x
    neweq/=denominator
    return simplify(neweq;expand=true)
end

integrateTaylorPolynomials (generic function with 1 method)

In [2]:
function ignoreHigherOrderTerms(eq::Num,x::Num,highestOrderForConsideration::Int)
    eq = simplify(eq;expand=true,polynorm=true)
    eqval=Symbolics.value(eq)


    # The above statement is to switch off the negative orders integral
    
    if Symbolics.isdiv(eqval)
        eq=eqval.num 
        denominator=eqval.den 
    else 
        denominator =1
    end
    highestOrder=Symbolics.degree(eq,x)

    highestOrderForConsideration > 0 || error("highestOrderForConsideration should be positive")
    highestOrderForConsideration<highestOrder || return eq/denominator
    tmpeq=eq
    for i in highestOrderForConsideration+1:highestOrder

        old_coef=Symbolics.coeff(eq,x^i)

        tmpeq-=old_coef*x^i
    end
    neweq=tmpeq/denominator
    return simplify(neweq; expand=true)
end
        

ignoreHigherOrderTerms (generic function with 1 method)

In [74]:
orderUtime=4
#orderUspace=8
#m = 15 # the midpoint to evaluate the operator at
L₁ₗ = 2 # the leftmost point for x1 direction
L₁ᵣ = 2 # the rightmost point
L₂ₗ = 2 # the leftmost point for t direction
L₂ᵣ = 2 # the rightmost point
L₁ₗ = 1 # the leftmost point for x1 direction
L₁ᵣ = 1 # the rightmost point
L₂ₗ = 1 # the leftmost point for t direction
L₂ᵣ = 1 # the rightmost point

orderStructure=L₁ₗ+L₁ᵣ # number of nodes minus one

@variables dx₁ dx₂   
@variables Cᵘ[1:orderUspace+1,1:orderUspace+1] # These are the derivatives at the node point xₘ, tₙ
#@variables x¹ t




#@variables ξ¹[1:L₁ₗ+L₁ᵣ+1]  τ[1:Lₜₗ+Lₜᵣ+1] # These are the node points in the spatial and time domain
#@variables ξ¹ₘ[-L₁ₗ:L₁ᵣ] τₙ[-L₁ₗ:L₁ᵣ] ρₘ[-L₁ₗ:L₁ᵣ] μₘ[-L₁ₗ:L₁ᵣ]
ξ¹ₘ = Symbolics.variables(:ξ¹ₘ,-L₁ₗ:L₁ᵣ) # nodes in space (x₁ direction)
ξ²ₘ = Symbolics.variables(:ξ²ₘ,-L₂ₗ:L₂ᵣ) 
uₘₙ= Symbolics.variables(:uₘₙ,-L₁ₗ:L₁ᵣ,-L₂ₗ:L₂ᵣ)
Aₘₙ= Symbolics.variables(:Aₘₙ,-L₁ₗ:L₁ᵣ,-L₂ₗ:L₂ᵣ)


u(dx₁,dx₂) = sum(sum(Cᵘ[i,j] * dx₁^(i-1) * dx₂^(j-1) / factorial(BigInt(i-1)) / factorial(BigInt(j-1)) for i in 1:orderUspace+1) for j in 1:orderUspace+1)

u (generic function with 1 method)

# First step


In [75]:
@variables X,Y #
∂y=Differential(Y)
∂x=Differential(X)
∂x2 = ∂x^2
∂y2 = ∂y^2

Differential(Y) ∘ Differential(Y)

In [76]:
simplify(u(X,Y))

Cᵘ[1, 1] + Cᵘ[1, 2]*Y + Cᵘ[2, 1]*X + (1//2)*Cᵘ[1, 3]*(Y^2) + Cᵘ[2, 2]*X*Y + (1//2)*Cᵘ[3, 1]*(X^2) + (1//6)*Cᵘ[1, 4]*(Y^3) + (1//2)*Cᵘ[2, 3]*X*(Y^2) + (1//2)*Cᵘ[3, 2]*(X^2)*Y + (1//6)*Cᵘ[4, 1]*(X^3) + (1//24)*Cᵘ[1, 5]*(Y^4) + (1//6)*Cᵘ[2, 4]*X*(Y^3) + (1//4)*Cᵘ[3, 3]*(X^2)*(Y^2) + (1//6)*Cᵘ[4, 2]*(X^3)*Y + (1//24)*Cᵘ[5, 1]*(X^4) + (1//120)*Cᵘ[1, 6]*(Y^5) + (1//24)*Cᵘ[2, 5]*X*(Y^4) + (1//12)*Cᵘ[3, 4]*(X^2)*(Y^3) + (1//12)*Cᵘ[4, 3]*(X^3)*(Y^2) + (1//24)*Cᵘ[5, 2]*(X^4)*Y + (1//120)*Cᵘ[6, 1]*(X^5) + (1//720)*Cᵘ[1, 7]*(Y^6) + (1//120)*Cᵘ[2, 6]*X*(Y^5) + (1//48)*Cᵘ[3, 5]*(X^2)*(Y^4) + (1//36)*Cᵘ[4, 4]*(X^3)*(Y^3) + (1//48)*Cᵘ[5, 3]*(X^4)*(Y^2) + (1//120)*Cᵘ[6, 2]*(X^5)*Y + (1//720)*Cᵘ[7, 1]*(X^6) + (1//5040)*Cᵘ[1, 8]*(Y^7) + (1//720)*Cᵘ[2, 7]*X*(Y^6) + (1//240)*Cᵘ[3, 6]*(X^2)*(Y^5) + (1//144)*Cᵘ[4, 5]*(X^3)*(Y^4) + (1//144)*Cᵘ[5, 4]*(X^4)*(Y^3) + (1//240)*Cᵘ[6, 3]*(X^5)*(Y^2) + (1//720)*Cᵘ[7, 2]*(X^6)*Y + (1//5040)*Cᵘ[8, 1]*(X^7) + (1//40320)*Cᵘ[1, 9]*(Y^8) + (1//5040)*Cᵘ[2, 8]*X*(Y^7) + (1//

# second step


In [77]:
∇u = expand_derivatives(∂x2(u(X,Y)))+expand_derivatives(∂y2(u(X,Y)))

Cᵘ[1, 3] + Cᵘ[3, 1] + Cᵘ[1, 4]*Y + Cᵘ[2, 3]*X + Cᵘ[3, 2]*Y + Cᵘ[4, 1]*X + (1//2)*Cᵘ[1, 5]*(Y^2) + Cᵘ[2, 4]*X*Y + (1//2)*Cᵘ[3, 3]*(X^2) + (1//2)*Cᵘ[3, 3]*(Y^2) + Cᵘ[4, 2]*X*Y + (1//2)*Cᵘ[5, 1]*(X^2) + (1//6)*Cᵘ[1, 6]*(Y^3) + (1//2)*Cᵘ[2, 5]*X*(Y^2) + (1//2)*Cᵘ[3, 4]*(X^2)*Y + (1//6)*Cᵘ[3, 4]*(Y^3) + (1//6)*Cᵘ[4, 3]*(X^3) + (1//2)*Cᵘ[4, 3]*X*(Y^2) + (1//2)*Cᵘ[5, 2]*(X^2)*Y + (1//6)*Cᵘ[6, 1]*(X^3) + (1//24)*Cᵘ[1, 7]*(Y^4) + (1//6)*Cᵘ[2, 6]*X*(Y^3) + (1//4)*Cᵘ[3, 5]*(X^2)*(Y^2) + (1//24)*Cᵘ[3, 5]*(Y^4) + (1//6)*Cᵘ[4, 4]*(X^3)*Y + (1//6)*Cᵘ[4, 4]*X*(Y^3) + (1//24)*Cᵘ[5, 3]*(X^4) + (1//4)*Cᵘ[5, 3]*(X^2)*(Y^2) + (1//6)*Cᵘ[6, 2]*(X^3)*Y + (1//24)*Cᵘ[7, 1]*(X^4) + (1//120)*Cᵘ[1, 8]*(Y^5) + (1//24)*Cᵘ[2, 7]*X*(Y^4) + (1//12)*Cᵘ[3, 6]*(X^2)*(Y^3) + (1//120)*Cᵘ[3, 6]*(Y^5) + (1//12)*Cᵘ[4, 5]*(X^3)*(Y^2) + (1//24)*Cᵘ[4, 5]*X*(Y^4) + (1//24)*Cᵘ[5, 4]*(X^4)*Y + (1//12)*Cᵘ[5, 4]*(X^2)*(Y^3) + (1//120)*Cᵘ[6, 3]*(X^5) + (1//12)*Cᵘ[6, 3]*(X^3)*(Y^2) + (1//24)*Cᵘ[7, 2]*(X^4)*Y + (1//120)*Cᵘ[8, 1]*(X^5) + 

In [78]:
@variables Δx
f1_spaceX=integrateTaylorPolynomials(∇u,X)
f2_spaceX=integrateTaylorPolynomials(f1_spaceX,X)
f2_spaceX_right=substitute(f2_spaceX,Dict([X=>Δx]))
f2_spaceX_left=substitute(f2_spaceX,Dict([X=>(-Δx)]))
ϕ_deriv_right=-1//Δx
ϕ_deriv_left = 1//Δx
desired_value_in_spaceX=-(ϕ_deriv_right*f2_spaceX_right-ϕ_deriv_left*f2_spaceX_left)
desired_value_in_spaceX=simplify(desired_value_in_spaceX;expand=true)

Cᵘ[1, 3]*Δx + Cᵘ[3, 1]*Δx + Cᵘ[1, 4]*Y*Δx + Cᵘ[3, 2]*Y*Δx + (1//2)*Cᵘ[1, 5]*(Y^2)*Δx + (1//2)*Cᵘ[3, 3]*(Y^2)*Δx + (1//12)*Cᵘ[3, 3]*(Δx^3) + (1//12)*Cᵘ[5, 1]*(Δx^3) + (1//6)*Cᵘ[1, 6]*(Y^3)*Δx + (1//6)*Cᵘ[3, 4]*(Y^3)*Δx + (1//12)*Cᵘ[3, 4]*Y*(Δx^3) + (1//12)*Cᵘ[5, 2]*Y*(Δx^3) + (1//24)*Cᵘ[1, 7]*(Y^4)*Δx + (1//24)*Cᵘ[3, 5]*(Y^4)*Δx + (1//24)*Cᵘ[3, 5]*(Y^2)*(Δx^3) + (1//24)*Cᵘ[5, 3]*(Y^2)*(Δx^3) + (1//360)*Cᵘ[5, 3]*(Δx^5) + (1//360)*Cᵘ[7, 1]*(Δx^5) + (1//120)*Cᵘ[1, 8]*(Y^5)*Δx + (1//120)*Cᵘ[3, 6]*(Y^5)*Δx + (1//72)*Cᵘ[3, 6]*(Y^3)*(Δx^3) + (1//72)*Cᵘ[5, 4]*(Y^3)*(Δx^3) + (1//360)*Cᵘ[5, 4]*Y*(Δx^5) + (1//360)*Cᵘ[7, 2]*Y*(Δx^5) + (1//720)*Cᵘ[1, 9]*(Y^6)*Δx + (1//720)*Cᵘ[3, 7]*(Y^6)*Δx + (1//288)*Cᵘ[3, 7]*(Y^4)*(Δx^3) + (1//288)*Cᵘ[5, 5]*(Y^4)*(Δx^3) + (1//720)*Cᵘ[5, 5]*(Y^2)*(Δx^5) + (1//720)*Cᵘ[7, 3]*(Y^2)*(Δx^5) + (1//20160)*Cᵘ[7, 3]*(Δx^7) + (1//20160)*Cᵘ[9, 1]*(Δx^7) + (1//5040)*Cᵘ[3, 8]*(Y^7)*Δx + (1//1440)*Cᵘ[3, 8]*(Y^5)*(Δx^3) + (1//1440)*Cᵘ[5, 6]*(Y^5)*(Δx^3) + (1//2160)*Cᵘ[5, 6]*(Y^3)

In [79]:
@variables Δy
f1_spaceY=integrateTaylorPolynomials(desired_value_in_spaceX,Y)
f2_spaceY=integrateTaylorPolynomials(f1_spaceY,Y)
f2_spaceY_right=substitute(f2_spaceY,Dict([Y=>Δy]))
f2_spaceY_left=substitute(f2_spaceY,Dict([Y=>(-Δy)]))
ϕ_deriv_right=-1//Δy
ϕ_deriv_left = 1//Δy
desired_value_in_spaceY=-(ϕ_deriv_right*f2_spaceY_right-ϕ_deriv_left*f2_spaceY_left)
desired_value_in_spaceY=simplify(desired_value_in_spaceY;expand=true)

Cᵘ[1, 3]*Δx*Δy + Cᵘ[3, 1]*Δx*Δy + (1//12)*Cᵘ[1, 5]*Δx*(Δy^3) + (1//12)*Cᵘ[3, 3]*(Δx^3)*Δy + (1//12)*Cᵘ[3, 3]*Δx*(Δy^3) + (1//12)*Cᵘ[5, 1]*(Δx^3)*Δy + (1//360)*Cᵘ[1, 7]*Δx*(Δy^5) + (1//144)*Cᵘ[3, 5]*(Δx^3)*(Δy^3) + (1//360)*Cᵘ[3, 5]*Δx*(Δy^5) + (1//360)*Cᵘ[5, 3]*(Δx^5)*Δy + (1//144)*Cᵘ[5, 3]*(Δx^3)*(Δy^3) + (1//360)*Cᵘ[7, 1]*(Δx^5)*Δy + (1//20160)*Cᵘ[1, 9]*Δx*(Δy^7) + (1//4320)*Cᵘ[3, 7]*(Δx^3)*(Δy^5) + (1//20160)*Cᵘ[3, 7]*Δx*(Δy^7) + (1//4320)*Cᵘ[5, 5]*(Δx^5)*(Δy^3) + (1//4320)*Cᵘ[5, 5]*(Δx^3)*(Δy^5) + (1//20160)*Cᵘ[7, 3]*(Δx^7)*Δy + (1//4320)*Cᵘ[7, 3]*(Δx^5)*(Δy^3) + (1//20160)*Cᵘ[9, 1]*(Δx^7)*Δy + (1//241920)*Cᵘ[3, 9]*(Δx^3)*(Δy^7) + (1//1814400)*Cᵘ[3, 9]*Δx*(Δy^9) + (1//129600)*Cᵘ[5, 7]*(Δx^5)*(Δy^5) + (1//241920)*Cᵘ[5, 7]*(Δx^3)*(Δy^7) + (1//241920)*Cᵘ[7, 5]*(Δx^7)*(Δy^3) + (1//129600)*Cᵘ[7, 5]*(Δx^5)*(Δy^5) + (1//1814400)*Cᵘ[9, 3]*(Δx^9)*Δy + (1//241920)*Cᵘ[9, 3]*(Δx^7)*(Δy^3) + (1//7257600)*Cᵘ[5, 9]*(Δx^5)*(Δy^7) + (1//21772800)*Cᵘ[5, 9]*(Δx^3)*(Δy^9) + (1//7257600)*Cᵘ[7, 7]*(Δx^7

In [80]:
innerProduct=0
for i¹ in 1:L₁ₗ+L₁ᵣ+1
    dx¹=Δx*(i¹-L₁ₗ-1)
    for i² in 1:L₂ₗ+L₂ᵣ+1
        dx² = Δy*(i²-L₂ₗ-1)
        innerProduct+=Aₘₙ[i¹,i²]*u(dx¹,dx²)
    end
end
difference_between_matrix_strongform=simplify(innerProduct-desired_value_in_spaceY;expand=true)
Symbolics.coeff(difference_between_matrix_strongform,Cᵘ[1,3])

-Δx*Δy + (1//2)*Aₘₙ₀ˏ₁*(Δy^2) + (1//2)*Aₘₙ₀ˏ₋₁*(Δy^2) + (1//2)*Aₘₙ₁ˏ₁*(Δy^2) + (1//2)*Aₘₙ₁ˏ₋₁*(Δy^2) + (1//2)*Aₘₙ₋₁ˏ₁*(Δy^2) + (1//2)*Aₘₙ₋₁ˏ₋₁*(Δy^2)

In [81]:
hierarchical_equations=[]
for i¹ in 1:L₁ₗ+L₁ᵣ+1   
    for i² in 1:L₂ₗ+L₂ᵣ+1 
        push!(hierarchical_equations,Symbolics.coeff(difference_between_matrix_strongform,Cᵘ[i¹,i²])~0)
    end
end
newA=reshape(Symbolics.solve_for(hierarchical_equations,Aₘₙ),(L₁ₗ+L₁ᵣ+1,L₂ₗ+L₂ᵣ+1 ))
simplify(newA[1,3])

(-(Δx^2) - (Δy^2)) / (-12Δx*Δy)

In [85]:
for i¹ in 1:L₁ₗ+L₁ᵣ+1
    for i² in 1:L₂ₗ+L₂ᵣ+1 
        display(simplify(newA[i¹,i²]*Δx*Δy*12;expand=true))
    end
end

Δx^2 + Δy^2

-(2//1)*(Δx^2) + (10//1)*(Δy^2)

Δx^2 + Δy^2

(10//1)*(Δx^2) - (2//1)*(Δy^2)

-(20//1)*(Δx^2) - (20//1)*(Δy^2)

(10//1)*(Δx^2) - (2//1)*(Δy^2)

Δx^2 + Δy^2

-(2//1)*(Δx^2) + (10//1)*(Δy^2)

Δx^2 + Δy^2